<a href="https://colab.research.google.com/github/nbilgee/git_exercise/blob/main/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'''Sequence to sequence example in Keras (character-level).

This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

# Summary of the algorithm:

- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

# Data download:

English to French sentence pairs.
https://www.manythings.org/anki/fra-eng.zip

Lots of neat sentence pairs datasets can be found at:
https://www.manythings.org/anki/

# References:

- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078

# Problem 1 Machine translation execution and code reading

In [ ]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

- line 1-4: Import library

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

- Line 1-4: hyperparameter settings

In [ ]:
data_path = 'fra2.txt'
lines = open(data_path, encoding='utf8').read().split('\n')
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
for line in lines[: min(num_samples, len(lines) - 1)]:
  input_text, target_text = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 90
Max sequence length for inputs: 16
Max sequence length for outputs: 53


- Line 1: get the data path
- Line 2: opening file
- Line 3 to 6: define variable input and target test and characters
- Line 8: loop along the length of the file string
- Line 9: Divide input text and target text
- Line 10 to 11: save to list
- Line 12 to 17: loop along the lenght of input text and add in input,target characters
- Line 18 to 19: sorted the characters
- Line 20-21: get lenght characters
- Line 22: get lenght max sequence
- Line 23: return values

In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

- Line 1 to 3: assign an index to each charectar. But the characters do not match

In [ ]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
# print("endoder input data:\n{}".format(encoder_input_data[:][1]))
# print("decoder input data:\n{}".format(decoder_input_data[:][1]))
# print("decoder target data:\n{}".format(decoder_target_data[:][1]))
#Zeros

- line 1 to 3: definition encoder data and decoder data

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.

    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.

        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.


print(f"decoder target data shape: {decoder_target_data.shape}")
print(f"encoder input data shape: {encoder_input_data.shape}")

decoder target data shape: (10000, 53, 90)
encoder input data shape: (10000, 16, 70)


- Line 1 to 8: give value in encoder input data and decoder target data

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


- Line 1 to 4: Define an input sequence and process it.
- Line 5 to 6: discard `encoder_outputs` and only keep the states.
- Line 7 to 8: Set up the decoder, using `encoder_states` as initial state.
- Line 9 to 15: We set up our decoder to return full output sequences, and to return internal states as well. We don't use the return states in the training model, but we will use them in inference.

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)
model.save('s2s.h5')

Epoch 1/100
125/125 [==============================] - 76s 582ms/step - loss: 1.2518 - val_loss: 1.3571
Epoch 2/100
125/125 [==============================] - 78s 623ms/step - loss: 1.1648 - val_loss: 1.3252
Epoch 3/100
125/125 [==============================] - 76s 607ms/step - loss: 1.1430 - val_loss: 1.3220
Epoch 4/100
125/125 [==============================] - 83s 664ms/step - loss: 1.1232 - val_loss: 1.3059
Epoch 5/100
125/125 [==============================] - 79s 634ms/step - loss: 1.1051 - val_loss: 1.2682
Epoch 6/100
125/125 [==============================] - 80s 640ms/step - loss: 1.0850 - val_loss: 1.2648
Epoch 7/100
125/125 [==============================] - 82s 656ms/step - loss: 1.0671 - val_loss: 1.2481
Epoch 8/100
125/125 [==============================] - 82s 662ms/step - loss: 1.0504 - val_loss: 1.2229
Epoch 9/100
125/125 [==============================] - 84s 675ms/step - loss: 1.0352 - val_loss: 1.2063
Epoch 10/100
125/125 [==============================] - 80s 639m

- line 1: Define the model that will turn `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
- line 2 to 3: Run training
- line 4: Sava model

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.load_weights('./s2s.h5')

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model( [decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


- line 1 to 8: inference mode (sampling).Here's the drill:
1) encode input and retrieve initial decoder state
2) run one step of decoder with this initial state and a "start of sequence" token as target.Output will be the next target token
3) Repeat with the current target token and current states
- Line 10 to 11:  Reverse-lookup token index to decode sequences back to something readable.

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        # Update states
        states_value = [h, c]
    return decoded_sentence

- Line 1: Define sampling models
- Line 2 to 3: Encode the input as state vectors.
- Line 4 to 5: Generate empty target sequence of length 1.
- Line 6 to 7: Populate the first character of target sequence with the start character.
- Line 8 to 14: Sampling loop for a batch of sequences (to simplify, here we assume a batch of size 1).
- Line 15 to 18: Sample a token
- Line 19 to 24: Exit condition: either hit max length or find stop character.
- Line 25 to 27: Update the target sequence (of length 1).
- Line 28 to 29: Update states
- Line 30: return value

In [ ]:
input_sentence = "How are you?"
test_sentence_tokenized = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
  test_sentence_tokenized[0, t, input_token_index[char]] = 1.
print(input_sentence)
print(decode_sequence(test_sentence_tokenized))

How are you?
1/1 [==============================] - 0s 21ms/step
Wie geht es geheneeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee


In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

- Line 1: looping 100
- Line 2 to 5: Take one sequence (part of the training test) for trying out decoding.

# Problem 2 Running a trained model for image captioning

#### 1. Clone the repositories
```bash
git clone https://github.com/pdollar/coco.git
cd coco/PythonAPI/
make
python setup.py build
python setup.py install
cd ../../
git clone https://github.com/yunjey/pytorch-tutorial.git
cd pytorch-tutorial/tutorials/03-advanced/image_captioning/
```

#### 2. Download the dataset

```bash
pip install -r requirements.txt
chmod +x download.sh
./download.sh
```

#### 3. Preprocessing

```bash
python build_vocab.py   
python resize.py
```

#### 4. Train the model

```bash
python train.py    
```

#### 5. Test the model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! python /content/gdrive/MyDrive/Dive_into_code/Assignments/Time_series_data_processing/image_captioning/sample.py \
    --image='/content/gdrive/MyDrive/Dive_into_code/Assignments/Time_series_data_processing/image_captioning/png/example.png' \
    --vocab_path='/content/gdrive/MyDrive/Dive_into_code/Assignments/Time_series_data_processing/image_captioning/models/vocab.pkl' \
    --decoder_path='/content/gdrive/MyDrive/Dive_into_code/Assignments/Time_series_data_processing/image_captioning/models/decoder-5-3000.pkl' \
    --encoder_path='/content/gdrive/MyDrive/Dive_into_code/Assignments/Time_series_data_processing/image_captioning/models/encoder-5-3000.pkl'

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
<start> a group of giraffes standing next to each other . <end>


# Question 3 Investigate what to do if you want to run with Keras

If you want to run a PyTorch implementation using Keras and also utilize pre-trained weights from PyTorch, there are several steps you can follow. Please note that direct conversion of models or weights between PyTorch and Keras is not possible due to differences in their underlying frameworks. However, you can still achieve the desired outcome by adapting the model architecture and transferring the weights manually.

Here's a step-by-step guide on how you can proceed:

Understand the model architecture: Begin by studying the PyTorch model architecture that you want to convert to Keras. This includes understanding the layers, activation functions, and any specific modifications that might have been made to the original architecture.

Implement the Keras model: Start by creating a new model in Keras with the same architecture as the PyTorch model. You can use the Keras Sequential API or the functional API, depending on the complexity of the model. Make sure to define the same layers and activations as in the PyTorch model. Pay attention to any differences in layer ordering, padding, or parameter initialization between PyTorch and Keras.

Load the pre-trained PyTorch weights: Since direct conversion of weights is not possible, you need to load the pre-trained PyTorch weights separately and manually transfer them to the Keras model. First, save the PyTorch model's state dictionary using torch.save(model.state_dict(), 'model_weights.pth').

Adapt the weights for Keras: To adapt the PyTorch weights for use in Keras, you need to iterate through the layers of both models and manually transfer the weights. You can load the saved PyTorch weights using state_dict = torch.load('model_weights.pth'). Then, for each layer in the Keras model, extract the corresponding weights from the PyTorch state dictionary and set them in the Keras model using layer.set_weights(weights). Pay attention to the shape and ordering of the weights in PyTorch and Keras, as they might differ.

Freeze layers if necessary: If you want to keep some layers frozen (not trainable) in Keras, you can set the trainable attribute of those layers to False. This is particularly useful when using pre-trained weights, as you might want to freeze the initial layers and only fine-tune the later layers.

Compile and train the Keras model: Once the model architecture and weights have been adapted, compile the Keras model by specifying the optimizer, loss function, and any metrics you want to track during training. Then, you can train the model on your dataset using the Keras API, providing the input data and corresponding labels.

It's important to note that despite following these steps, the performance of the model may vary between PyTorch and Keras due to differences in optimization algorithms and numerical precision. It is advisable to evaluate and fine-tune the Keras model to ensure comparable performance to the original PyTorch implementation.

By following these steps, you should be able to convert a PyTorch implementation to Keras, including the usage of pre-trained weights from PyTorch.